<a href="https://colab.research.google.com/github/Tchacor/mao_na_roda/blob/main/agente_mao_na_roda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
%pip -q install Pillow

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai # Importa o pacote google.genai
from google.genai import types # Para criar conteúdos (Content e Part)

In [ ]:
# Inicializa o cliente.

try:
    client = genai.Client()
    print("Cliente google.genai.Client inicializado com sucesso.")
except Exception as e:
    print(f"Erro ao inicializar google.genai.Client: {e}")
    client = None

MODEL_ID = "gemini-2.0-flash"

Cliente google.genai.Client inicializado com sucesso.


In [ ]:
# Instalar Framework ADK de agentes do Google
%pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from datetime import date
from PIL import Image # Para manipulação de imagens
import pathlib # Para manipulação de caminhos de arquivo
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import warnings

warnings.filterwarnings("ignore") # Suprime avisos

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
################################################
# --- Agente Mão na Roda --- #
################################################
SYSTEM_INSTRUCTION="""
Você é o "Mão na Roda", um assistente de IA amigável, paciente e especialista em pequenos reparos domésticos e projetos de faça-você-mesmo.
Sua missão é ajudar usuários sem experiência a resolverem problemas comuns em suas casas, de forma segura e didática.

**Suas diretrizes são:**

1.  **Tom Conversacional e Amigável:** Mantenha uma conversa natural, empática e encorajadora. Use linguagem simples e evite jargões técnicos excessivos.
Se precisar usar um termo técnico, explique-o.
2.  **Diagnóstico Interativo (Uma Pergunta por Vez):** Quando um usuário descrever um problema, faça perguntas de acompanhamento para entender completamente
a situação. **CRÍTICO: Faça apenas UMA pergunta de esclarecimento por vez. Espere a resposta do usuário antes de fazer a próxima pergunta ou oferecer uma
solução.** Não presuma as respostas. Se você tiver várias perguntas em mente para o diagnóstico, pergunte a mais importante ou a primeira lógica primeiro,
aguarde a resposta, e então decida qual será sua próxima pergunta. O objetivo é guiar o usuário passo a passo no diagnóstico.
3.  **Instruções Passo a Passo Claras:** Forneça instruções claras, concisas e em formato de passo a passo. Numere os passos.
4.  **Segurança em Primeiro Lugar:** SEMPRE inclua avisos de segurança relevantes ANTES de qualquer instrução que envolva riscos (eletricidade, ferramentas
cortantes, produtos químicos, altura, etc.). Se o risco for alto ou o usuário parecer inseguro, sugira fortemente chamar um profissional qualificado.
5.  **Sugestão de Materiais e Ferramentas:** Após o diagnóstico e antes das instruções, liste os materiais e ferramentas provavelmente necessários.
6.  **Manutenção de Contexto:** Lembre-se do histórico da conversa para que o diálogo seja fluido e coerente.
7.  **Foco em "Faça Você Mesmo":** Seu objetivo principal é capacitar o usuário.
8.  **Saber Limites:** Se o problema descrito parecer muito complexo, perigoso para um leigo, ou fora do escopo de pequenos reparos domésticos (ex: problemas
estruturais, grandes reformas), explique educadamente que o problema excede sua capacidade de ajudar com segurança e recomende que o usuário procure um profissional qualificado.
9.  **Incentivo e Paciência:** Muitos usuários podem ser iniciantes. Seja paciente, responda a todas as perguntas e incentive-ios.
10. **Formato da Resposta:**
    * Use negrito para destacar termos importantes ou cabeçalhos de seção (Ex: **Diagnóstico:**, **Materiais Necessários:**, **Passo a Passo:**, **⚠️ Dicas de Segurança: ⚠️**).
    * Use listas numeradas para instruções.
    * Use emojis para tornar a conversa mais leve e visual quando apropriado (mas sem exagero).
    * Quando for dar algumas intruções, faça isso de forma pausada para que o texto não fique extenso para o usuário. *
11. **Uso de Imagens para Diagnóstico:**
    * Se o problema for visual ou difícil de descrever apenas com texto (ex: uma peça quebrada específica, um tipo de conexão, uma mancha, um código de erro
    em um painel), **pergunte ativamente ao usuário se ele pode enviar uma foto do problema.** Ex: "Isso parece algo que uma foto poderia ajudar a esclarecer.
    Você conseguiria me enviar uma imagem do [objeto/problema]?"
    * Ao analisar uma imagem fornecida pelo usuário, use-a em conjunto com a descrição textual para:
        * Identificar melhor o problema visualmente.
        * Tentar reconhecer modelos de equipamentos, peças, conexões ou ferramentas.
        * Observar detalhes que o usuário pode não ter mencionado.
        * Ajudar a determinar os materiais ou ferramentas corretos.
    * Após analisar a imagem, formule suas perguntas ou instruções subsequentes com base nas informações combinadas do texto e da imagem.
    * Se a imagem não for clara, for irrelevante ou não ajudar no diagnóstico, explique isso educadamente e continue com perguntas textuais para obter mais
    informações. Não finja que entendeu algo da imagem se não for o caso.
12. **Uso Opcional da Busca na Web:** Se você precisar de informações muito recentes (posteriores à sua data de corte de conhecimento) ou detalhes específicos
sobre produtos, manuais, ou lojas, você pode e deve usar a ferramenta de busca na web para encontrar essas informações antes de responder. Use-a apenas quando
for realmente necessário para fornecer uma resposta precisa e útil.

**Exemplo de interação inicial desejada (para guiar o modelo):**

Usuário: "Minha torneira da cozinha está pingando."

Você (Mão na Roda): "Olá! Entendo, uma torneira pingando pode ser bem chata! 💧 Para te ajudar melhor, preciso de um detalhe primeiro: essa torneira é de
que tipo? (Ex: monocomando, com dois registros separados para água quente e fria, de parede, de bancada?)"
(Após a resposta do usuário, você faria a próxima pergunta)

**Exemplo de interação para instruções muito longas (para guiar o modelo):**

Usuário: "Quero trocar a lâmpada da minha sala."

Você (Mão na Roda): "Ótimo! Vamos lá. Primeiro, desligue a luz e a tomada da lâmpada. Depois, retire a lâmpada antiga.
Você (Mão na Roda): "Pergunta se o usuário precisa de ajuda para retirar a lâmpada antiga. Se sim, dê instruções detalhadas. Se não, continue."
Você (Mão na Roda): "Agora, coloque a nova lâmpada no lugar e aperte para encaixar bem. Por fim, ligue a luz e a tomada."

**Exemplo de interação com imagem (para guiar o modelo):**

Usuário: "Acho que o cano debaixo da minha pia está vazando, mas não sei qual peça é."
Você (Mão na Roda): "Entendo, vazamentos podem ser complicados. Uma foto poderia me ajudar muito a identificar a peça e o local exato do vazamento.
Você conseguiria tirar uma foto da área debaixo da pia e me enviar?"
(Se o usuário enviar a foto)
Você (Mão na Roda): "Obrigado pela foto! Analisando a imagem, vejo que [descrição do que vê na imagem e como isso ajuda]..."

Lembre-se, seu objetivo é ser o "Faz-Tudo Digital" de confiança do usuário, guiando-o de forma clara e uma pergunta por vez durante o diagnóstico!
"""

In [ ]:
if client is None:
    print("ERRO CRÍTICO: O cliente google.genai.Client não foi inicializado. Verifique a Célula 4 e a configuração da API Key.")
else:
    generation_config_params = {
        "temperature": 0.7,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 8192,
    }

    safety_settings_params = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    ]

    chat_history = []

    # MENSAGENS INICIAIS FORMATADAS
    display(Markdown("### 🛠️ Olá! Eu sou o **Mão na Roda**, seu assistente digital para reparos domésticos!"))
    display(Markdown("> 💡 **Dica:** Se eu pedir uma foto, ou se você quiser me enviar uma, digite `/enviarimagem` e pressione Enter. Depois, na próxima linha, digite o caminho completo para o arquivo da imagem."))
    display(Markdown("--- \nComo posso te ajudar hoje?"))


    while True:
        # O prompt do input() não pode ser formatado com Markdown diretamente na mesma linha.
        # A formatação da entrada do usuário será feita *após* o input.
        user_input_prompt = "Você: "
        user_text_input = ""
        image_to_send = None # Objeto PIL.Image
        image_path_for_display = None # Para exibir o nome do arquivo

        try:
            user_input_line1 = input(user_input_prompt)
        except EOFError:
            display(Markdown("> Mão na Roda: Parece que a conexão foi interrompida. Até a próxima!"))
            break
        except KeyboardInterrupt:
            display(Markdown("\n> Mão na Roda: Entendido, encerrando o chat. Até logo! 👋"))
            break

        if user_input_line1.lower() in ["sair", "exit", "fim", "tchau"]:
            display(Markdown("> Mão na Roda: Até a próxima! Se precisar de mais alguma ajuda, é só chamar! 👋"))
            break

        if user_input_line1.lower() == "/enviarimagem":
            image_path_str = input("Caminho da imagem: ")
            try:
                image_path = pathlib.Path(image_path_str)
                if not image_path.is_file():
                    display(Markdown(f"> Mão na Roda: Caminho inválido ou não é um arquivo (`{image_path_str}`). Tente novamente."))
                    print("-" * 30)
                    continue
                if image_path.suffix.lower() not in [".jpg", ".jpeg", ".png", ".webp"]:
                    display(Markdown("> Mão na Roda: Formato de imagem não suportado. Use JPG, PNG ou WEBP."))
                    print("-" * 30)
                    continue

                # Não exibir "processando" aqui, faremos isso junto com a entrada do usuário
                image_to_send = Image.open(image_path)
                image_path_for_display = image_path.name # Guardar nome para exibição
                user_text_accompanying_image = input("Você (opcional, descreva a imagem ou faça sua pergunta): ")
                if user_text_accompanying_image:
                    user_text_input = user_text_accompanying_image
            except FileNotFoundError:
                display(Markdown(f"> Mão na Roda: Arquivo de imagem não encontrado em `{image_path_str}`. Verifique e tente novamente."))
                print("-" * 30)
                continue
            except Exception as e:
                display(Markdown(f"> Mão na Roda: Erro ao carregar a imagem: {e}"))
                print("-" * 30)
                continue
        else:
            user_text_input = user_input_line1

        if not user_text_input and not image_to_send:
            display(Markdown("> Mão na Roda: Por favor, digite algo ou use `/enviarimagem`."))
            print("-" * 30)
            continue

        # EXIBIR A ENTRADA DO USUÁRIO FORMATADA
        entrada_formatada_md = "**Você:** "
        if image_path_for_display:
            entrada_formatada_md += f"[Imagem: `{image_path_for_display}`]"
            if user_text_input:
                entrada_formatada_md += f" {user_text_input}"
        elif user_text_input:
            entrada_formatada_md += user_text_input
        display(Markdown(entrada_formatada_md))


        # Construir as partes da mensagem do usuário atual
        current_user_message_parts = []
        if user_text_input:
            current_user_message_parts.append(types.Part(text=user_text_input))
        if image_to_send: # image_to_send é o objeto PIL.Image
            import io
            img_byte_arr = io.BytesIO()
            image_format = image_to_send.format if image_to_send.format else 'PNG'
            if image_format == 'JPEG': mime_type = 'image/jpeg'
            elif image_format == 'PNG': mime_type = 'image/png'
            elif image_format == 'WEBP': mime_type = 'image/webp'
            else:
                mime_type = 'image/png'
                image_format = 'PNG'
            image_to_send.save(img_byte_arr, format=image_format)
            img_bytes = img_byte_arr.getvalue()
            current_user_message_parts.append(types.Part(inline_data=types.Blob(mime_type=mime_type, data=img_bytes)))

        if not current_user_message_parts:
             display(Markdown("> Mão na Roda: Nenhuma mensagem para enviar. Tente novamente."))
             print("-" * 30)
             continue

        chat_history.append(types.Content(role="user", parts=current_user_message_parts))

        contents_for_api = [
            types.Content(role="user", parts=[types.Part(text=SYSTEM_INSTRUCTION + \
                                                         ("\n\nComece a conversa inicial." if len(chat_history) == 1 else "") )]),
        ]
        contents_for_api.extend(chat_history)

        # Indicador de que o modelo está processando (opcional)
        display(Markdown("> Mão na Roda está pensando... 🤔"))

        try:
            response = client.models.generate_content(
                model=f"models/{MODEL_ID}",
                contents=contents_for_api
            )

            model_response_text = ""
            if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
                 for part in response.candidates[0].content.parts:
                    if part.text:
                        model_response_text += part.text
            else:
                try: model_response_text = response.text
                except AttributeError: model_response_text = "Desculpe, não consegui gerar uma resposta em texto."

            if not model_response_text.strip() and response.prompt_feedback and response.prompt_feedback.block_reason:
                 model_response_text = f"A resposta foi bloqueada. Razão: {response.prompt_feedback.block_reason_message or response.prompt_feedback.block_reason}"

            # Limpar o "pensando..." - Não há uma forma fácil de remover output anterior no Colab sem usar libs mais complexas.
            # A nova resposta do Mão na Roda substituirá visualmente.
            # print("\033[F\033[K", end="") # Tentativa de limpar linha anterior (pode não funcionar bem no Colab)

            display(Markdown(f"**Mão na Roda:**")) # Label para a resposta
            display(to_markdown(model_response_text))

            chat_history.append(types.Content(role="model", parts=[types.Part(text=model_response_text)]))

        except Exception as e:
            error_detail = str(e)
            if hasattr(e, 'response') and hasattr(e.response, 'text'): error_detail += f"\nDetalhes da API: {e.response.text}"
            elif hasattr(e, 'message'): error_detail += f"\nDetalhes: {e.message}"

            display(Markdown(f"> **Mão na Roda:** Desculpe, não consegui processar sua solicitação."))
            display(Markdown(f"> ```\nErro: {error_detail}\n```")) # Erro em bloco de código
            display(Markdown(f"> Por favor, tente reformular sua pergunta ou problema."))

            if chat_history and chat_history[-1].role == "user": chat_history.pop()

        print("-" * 30) # Linha separadora

### 🛠️ Olá! Eu sou o **Mão na Roda**, seu assistente digital para reparos domésticos!

> 💡 **Dica:** Se eu pedir uma foto, ou se você quiser me enviar uma, digite `/enviarimagem` e pressione Enter. Depois, na próxima linha, digite o caminho completo para o arquivo da imagem.

--- 
Como posso te ajudar hoje?